In [19]:
import ads, re, csv
from nameparser import HumanName

In [2]:
def choose_best(names):
    lengths = [len(n.original) for n in names]
    longest = 0
    best = None
    for n, l in zip(names, lengths):
        if l > longest:
            best = n
    print(f'out of {[name.original for name in names]}, choose {n.original}')
    return n

def almost_equal(name1, name2):
    return name1.last.lower() == name2.last.lower() and name1.first[0].lower() == name2.first[0].lower()

In [3]:
ads.config.token = ''

In [4]:
# For this, go to ui.adsabs.harvard.edu, select papers, and export as AASTex

s = """
\bibitem[Korzoun et al.(2023)]{2023arXiv230809607K} Korzoun, N., Benbow, W., Brown, A., et al.\ 2023, arXiv:2308.09607. doi:10.48550/arXiv.2308.09607
\bibitem[Maire et al.(2022)]{2022SPIE12184E..8BM} Maire, J., Wright, S.~A., Holder, J., et al.\ 2022, \procspie, 12184, 121848B. doi:10.1117/12.2630772
\bibitem[Brown et al.(2021)]{2021arXiv211112771B} Brown, A.~M., Aronson, M.~L., Wright, S.~A., et al.\ 2021, arXiv:2111.12771. doi:10.48550/arXiv.2111.12771
\bibitem[Maire et al.(2020)]{2020SPIE11454E..3CM} Maire, J., Wright, S.~A., Werthimer, D., et al.\ 2020, \procspie, 11454, 114543C. doi:10.1117/12.2562786
\bibitem[Liu et al.(2020)]{2020SPIE11447E..7GL} Liu, W., Werthimer, D., Lee, R., et al.\ 2020, \procspie, 11447, 114477G. doi:10.1117/12.2561203
\bibitem[Brown et al.(2020)]{2020SPIE11445E..8BB} Brown, A.~M., Aronson, M.~L., Wright, S.~A., et al.\ 2020, \procspie, 11445, 114458B. doi:10.1117/12.2562985
\bibitem[Wright et al.(2019)]{2019BAAS...51g.264W} Wright, S., Antonio, F.~P., Aronson, M.~L., et al.\ 2019, \baas
\bibitem[Maire et al.(2018)]{2018SPIE10702E..5LM} Maire, J., Wright, S.~A., Cosens, M., et al.\ 2018, \procspie, 10702, 107025L. doi:10.1117/12.2314444
\bibitem[Wright et al.(2018)]{2018SPIE10702E..5IW} Wright, S.~A., Horowitz, P., Maire, J., et al.\ 2018, \procspie, 10702, 107025I. doi:10.1117/12.2314268
\bibitem[Cosens et al.(2018)]{2018SPIE10702E..5HC} Cosens, M., Maire, J., Wright, S.~A., et al.\ 2018, \procspie, 10702, 107025H. doi:10.1117/12.2314252
"""

In [25]:
bibcodes = re.findall('\{(2\S+)\}', s)

papers = [list(ads.SearchQuery(bibcode=bibcode))[0] for bibcode in bibcodes]

authors_aff = {}

for paper in papers:
    authors = paper.author
    affs = paper.aff
    for i in range(len(authors)):
        authors_aff[authors[i]]=affs[i]
authors_aff = dict(sorted(authors_aff.items()))

/home/wei/.conda/envs/py39/lib/python3.9/site-packages/ads/utils.py:33: UserWarning: You are lazy loading attributes via 'aff', and so are making multiple calls to the API. This will impact your overall rate limits.
  warnings.warn(


In [26]:
no_dups = {}
previous = HumanName('Hello Dotty')
to_compare = []
for author in authors_aff.keys():
    name = HumanName(author)
    if almost_equal(name, previous):
        print(f'{name}, {previous} being compared')
        to_compare.append(name)
    else:
        if len(to_compare) > 1:
            no_dups[choose_best(to_compare)]=authors_aff[author]
            to_compare = []
        else:
            no_dups[name]=authors_aff[author]
            to_compare = []
    previous = name

Franklin P. Antonio, Franklin Antonio being compared
Michael L Aronson, Michael Aronson being compared
Michael L. Aronson, Michael L Aronson being compared
out of ['Aronson, Michael L', 'Aronson, Michael L.'], choose Aronson, Michael L.
Wystan Benbow, Wynstan Benbow being compared
Aaron M. Brown, Aaron Brown being compared
Samuel A. Chaim-Weismann, Sam Chaim-Weismann being compared
Frank D. Drake, Frank Drake being compared
Andrew W. Howard, Andrew Howard being compared
Jérôme Maire, Jerome Maire being compared
Remington P. S. Stone, Remington Stone being compared
Richard R. Treffers, Richard Treffers being compared
James H. Wiley, James Wiley being compared
Shelley A. Wright, Shelley Wright being compared


In [27]:
f=open('coa.csv','w')
writer = csv.writer(f)
writer.writerow(['4', 'Name:', 'Organizational Affiliation', 'Optional  (email, Department)', 'Last Active'])
for auth in no_dups.keys():
    writer.writerow(['A', auth, no_dups[auth],' ',' '])
f.close()